In [1]:
import os, sys, time, glob, random, argparse
import numpy as np
from copy import deepcopy
import torch
import torch.nn as nn
from pathlib import Path
sys.path.append(str('/home/mzhang3/Data/2021NeurIPS_codes/NAS_Bench201/lib'))
sys.path.append(str('/home/mzhang3/Data/2021NeurIPS_codes/NAS_Bench201/configs'))
from config_utils import load_config, dict2config, configure2str
from datasets     import get_datasets, SearchDataset
from procedures   import prepare_seed, prepare_logger, save_checkpoint, copy_checkpoint, get_optim_scheduler
from utils        import get_model_infos, obtain_accuracy
from log_utils    import AverageMeter, time_string, convert_secs2time
from models       import get_cell_based_tiny_net, get_search_spaces
from nas_102_api  import NASBench102API as API

from weight_initializers import init_net




In [2]:
def search_func(xloader, network, criterion, scheduler, w_optimizer, a_optimizer, epoch_str, print_freq, logger):
    data_time, batch_time = AverageMeter(), AverageMeter()
    base_losses, base_top1, base_top5 = AverageMeter(), AverageMeter(), AverageMeter()
    arch_losses, arch_top1, arch_top5 = AverageMeter(), AverageMeter(), AverageMeter()
    network.train()
    end = time.time()   
    arch_pruned=deepcopy(network.module.arch_parameters.data)
    
    for step, (base_inputs, base_targets, arch_inputs, arch_targets) in enumerate(xloader):
        scheduler.update(None, 1.0 * step / len(xloader))
        base_targets = base_targets.cuda(non_blocking=True)
        arch_targets = arch_targets.cuda(non_blocking=True)
        # measure data loading time
        data_time.update(time.time() - end)

        # update the weights
        w_optimizer.zero_grad()
        _, logits = network(base_inputs)
        base_loss = criterion(logits, base_targets)
        #base_loss.backward()
        #torch.nn.utils.clip_grad_norm_(network.parameters(), 5)
        #w_optimizer.step()
        # record
        base_prec1, base_prec5 = obtain_accuracy(logits.data, base_targets.data, topk=(1, 5))
        base_losses.update(base_loss.item(),  base_inputs.size(0))
        base_top1.update  (base_prec1.item(), base_inputs.size(0))
        base_top5.update  (base_prec5.item(), base_inputs.size(0))

        # update the architecture-weight
        a_optimizer.zero_grad()
        
        input_dim = list(arch_inputs[0,:].shape)
        inputs = torch.ones([1] + input_dim).float().cuda(non_blocking=True)
        _, logits = network.forward(inputs)        
        
        arch_loss = torch.sum(logits)
        arch_loss.backward()
        #a_optimizer.step()
        # record

        num_batchs=10
        if step%num_batchs==0:         
            synflow=np.zeros(5)            
            availabe_edge=[]
            for edge in range(6):
                num_of_oper=0
                for i in range(5):
                    if arch_pruned[edge,i]> -float('Inf'):
                        num_of_oper +=1
                if num_of_oper>1:
                    availabe_edge.append(edge)
            edge_ind= np.random.randint(len(availabe_edge))   
            edge=availabe_edge[edge_ind]
            
            num_of_oper=0
            for i in range(5):
                if arch_pruned[edge,i]> -float('Inf'):
                    synflow[i]= network.module.arch_parameters.data[edge,i]*network.module.arch_parameters.grad.data[edge,i]
                    num_of_oper +=1
                else:
                    synflow[i]= float('Inf')
        else:
            synflow_new=np.zeros(5)
            for i in range(5):
                if arch_pruned[edge,i]> -float('Inf'):
                    synflow_new[i]= network.module.arch_parameters.data[edge,i]*network.module.arch_parameters.grad.data[edge,i]
            
            synflow = synflow + synflow_new    
                    
        if (step+1)%num_batchs==0:      
            if num_of_oper>1:
                ind=np.argmin(synflow)
                arch_pruned[edge,ind]=-float('Inf')
            logger.log('{:}'.format(api.query_by_arch(network.module.genotype_prune(arch_pruned))))  
            total_ope=0
            for edge in range(6):
                for i in range(5):
                    if arch_pruned[edge,i]> -float('Inf'):
                        total_ope +=1
            print(arch_pruned)
            print('---------------------------------------------')                                                
            if total_ope==6:
                break
                       
    return base_losses.avg, base_top1.avg, base_top5.avg

In [3]:
parser = argparse.ArgumentParser("DARTS First Order")
parser.add_argument('--data_path',          type=str,   default= '/data/mzhang3/ENNAS_master/data/cifar-10-batches-py', help='Path to dataset')
parser.add_argument('--dataset',            type=str,   default= 'cifar10', choices=['cifar10', 'cifar100', 'ImageNet16-120'], help='Choose between Cifar10/100 and ImageNet-16.')
# channels and number-of-cells
parser.add_argument('--search_space_name',  type=str,   default= 'nas-bench-102',help='The search space name.')
parser.add_argument('--max_nodes',          type=int,   default= 4 ,help='The maximum number of nodes.')
parser.add_argument('--channel',            type=int,   default= 16,help='The number of channels.')
parser.add_argument('--num_cells',          type=int,   default= 5, help='The number of cells in one stage.')
# architecture leraning rate
parser.add_argument('--arch_learning_rate', type=float, default=3e-2, help='learning rate for arch encoding')
parser.add_argument('--arch_weight_decay',  type=float, default=1e-3, help='weight decay for arch encoding')
# log
parser.add_argument('--workers',            type=int,   default=2,    help='number of data loading workers (default: 2)')
parser.add_argument('--save_dir',           type=str,   default='/output/search-cell-nas-bench-102-cifar10',help='Folder to save checkpoints and log.')
parser.add_argument('--arch_nas_dataset',   type=str,   default='/data/mzhang3/NSAS_Bench201/NAS-Bench-102-v1_0-e61699.pth',help='The path to load the architecture dataset (tiny-nas-benchmark).')
parser.add_argument('--print_freq',         type=int,   default=50,help='print frequency (default: 200)')
parser.add_argument('--rand_seed',          type=int,   default=5, help='manual seed')
args = parser.parse_args([])

In [5]:
args.save_dir= './Trained_supernet_Synflow_'+str(args.rand_seed)+args.save_dir

xargs=args
assert torch.cuda.is_available(), 'CUDA is not available.'
torch.cuda.set_device(0)
torch.backends.cudnn.enabled   = True
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
torch.set_num_threads( xargs.workers )
prepare_seed(xargs.rand_seed)
logger = prepare_logger(args)

train_data, valid_data, xshape, class_num = get_datasets(xargs.dataset, xargs.data_path, -1)
if xargs.dataset == 'cifar10' or xargs.dataset == 'cifar100':
    split_Fpath = '/home/mzhang3/Data/2021NeurIPS_codes/NAS_Bench201/configs/nas-benchmark/cifar-split.txt'
    cifar_split = load_config(split_Fpath, None, None)
    train_split, valid_split = cifar_split.train, cifar_split.valid
    logger.log('Load split file from {:}'.format(split_Fpath))
elif xargs.dataset.startswith('ImageNet16'):
    split_Fpath = 'configs/nas-benchmark/{:}-split.txt'.format(xargs.dataset)
    imagenet16_split = load_config(split_Fpath, None, None)
    train_split, valid_split = imagenet16_split.train, imagenet16_split.valid
    logger.log('Load split file from {:}'.format(split_Fpath))
else:
    raise ValueError('invalid dataset : {:}'.format(xargs.dataset))
config_path = '/home/mzhang3/Data/2021NeurIPS_codes/NAS_Bench201/configs/nas-benchmark/algos/DARTS.config'
config = load_config(config_path, {'class_num': class_num, 'xshape': xshape}, logger)
# To split data
train_data_v2 = deepcopy(train_data)
train_data_v2.transform = valid_data.transform
valid_data    = train_data_v2
search_data   = SearchDataset(xargs.dataset, train_data, train_split, valid_split)
# data loader
search_loader = torch.utils.data.DataLoader(search_data, batch_size=config.batch_size, shuffle=True , num_workers=xargs.workers, pin_memory=True)
valid_loader  = torch.utils.data.DataLoader(valid_data, batch_size=config.batch_size, sampler=torch.utils.data.sampler.SubsetRandomSampler(valid_split), num_workers=xargs.workers, pin_memory=True)
logger.log('||||||| {:10s} ||||||| Search-Loader-Num={:}, Valid-Loader-Num={:}, batch size={:}'.format(xargs.dataset, len(search_loader), len(valid_loader), config.batch_size))
logger.log('||||||| {:10s} ||||||| Config={:}'.format(xargs.dataset, config))

search_space = get_search_spaces('cell', xargs.search_space_name)
model_config = dict2config({'name': 'DARTS-V1', 'C': xargs.channel, 'N': xargs.num_cells,
                              'max_nodes': xargs.max_nodes, 'num_classes': class_num,
                              'space'    : search_space}, None)
search_model = get_cell_based_tiny_net(model_config)
logger.log('search-model :\n{:}'.format(search_model))

w_optimizer, w_scheduler, criterion = get_optim_scheduler(search_model.get_weights(), config)
a_optimizer = torch.optim.Adam(search_model.get_alphas(), lr=xargs.arch_learning_rate, betas=(0.5, 0.999), weight_decay=xargs.arch_weight_decay)


if xargs.arch_nas_dataset is None:
    api = None
else:
    api = API(xargs.arch_nas_dataset)
logger.log('{:} create API = {:} done'.format(time_string(), api))

last_info, model_base_path, model_best_path = logger.path('info'), logger.path('model'), logger.path('best')
network, criterion = torch.nn.DataParallel(search_model).cuda(), criterion.cuda()



Main Function with logger : Logger(dir=Trained_supernet_Synflow_5./Trained_supernet_Synflow_5/output/search-cell-nas-bench-102-cifar10, use-tf=False, writer=None)
Arguments : -------------------------------
arch_learning_rate : 0.03
arch_nas_dataset : /data/mzhang3/NSAS_Bench201/NAS-Bench-102-v1_0-e61699.pth
arch_weight_decay : 0.001
channel          : 16
data_path        : /data/mzhang3/ENNAS_master/data/cifar-10-batches-py
dataset          : cifar10
max_nodes        : 4
num_cells        : 5
print_freq       : 50
rand_seed        : 5
save_dir         : ./Trained_supernet_Synflow_5./Trained_supernet_Synflow_5/output/search-cell-nas-bench-102-cifar10
search_space_name : nas-bench-102
workers          : 2
Python  Version  : 3.6.8 |Anaconda, Inc.| (default, Dec 30 2018, 01:22:34)  [GCC 7.3.0]
Pillow  Version  : 7.1.2
PyTorch Version  : 1.6.0
cuDNN   Version  : 7605
CUDA available   : True
CUDA GPU numbers : 1
CUDA_VISIBLE_DEVICES : None
Files already downloaded and verified
Files already 

[2021-04-11 06:51:39] create API = NASBench102API(15625/15625 architectures) done


In [12]:
pa='/home/mzhang3/Data/2021NeurIPS_codes/NAS_Bench201/exps/algos/DARTS_V1_seed0/output/search-cell-nas-bench-102-cifar10/checkpoint/seed-0-basic-epoch-0-0.pth'

In [13]:
checkpoint  = torch.load(pa)

In [14]:
valid_accuracies = checkpoint['valid_accuracies']
search_model.load_state_dict( checkpoint['search_model'] )
w_scheduler.load_state_dict ( checkpoint['w_scheduler'] )
w_optimizer.load_state_dict ( checkpoint['w_optimizer'] )
a_optimizer.load_state_dict ( checkpoint['a_optimizer'] )

search_model.arch_parameters.data=nn.Parameter( 1e-3*torch.randn(6, 5)).cuda()#####3##

RuntimeError: Error(s) in loading state_dict for TinyNetworkDartsV1:
	Missing key(s) in state_dict: "arch_parameters", "stem.0.weight", "stem.1.weight", "stem.1.bias", "stem.1.running_mean", "stem.1.running_var", "cells.0.edges.1<-0.2.op.1.weight", "cells.0.edges.1<-0.2.op.2.running_mean", "cells.0.edges.1<-0.2.op.2.running_var", "cells.0.edges.1<-0.3.op.1.weight", "cells.0.edges.1<-0.3.op.2.running_mean", "cells.0.edges.1<-0.3.op.2.running_var", "cells.0.edges.2<-0.2.op.1.weight", "cells.0.edges.2<-0.2.op.2.running_mean", "cells.0.edges.2<-0.2.op.2.running_var", "cells.0.edges.2<-0.3.op.1.weight", "cells.0.edges.2<-0.3.op.2.running_mean", "cells.0.edges.2<-0.3.op.2.running_var", "cells.0.edges.2<-1.2.op.1.weight", "cells.0.edges.2<-1.2.op.2.running_mean", "cells.0.edges.2<-1.2.op.2.running_var", "cells.0.edges.2<-1.3.op.1.weight", "cells.0.edges.2<-1.3.op.2.running_mean", "cells.0.edges.2<-1.3.op.2.running_var", "cells.0.edges.3<-0.2.op.1.weight", "cells.0.edges.3<-0.2.op.2.running_me

In [15]:
start_time, search_time, epoch_time, total_epoch = time.time(), AverageMeter(), AverageMeter(), config.epochs + config.warmup
epoch=0

In [16]:
w_scheduler.update(epoch, 0.0)
need_time = 'Time Left: {:}'.format( convert_secs2time(epoch_time.val * (total_epoch-epoch), True) )
epoch_str = '{:03d}-{:03d}'.format(epoch, total_epoch)
min_LR    = min(w_scheduler.get_lr())
logger.log('\n[Search the {:}-th epoch] {:}, LR={:}'.format(epoch_str, need_time, min_LR))

search_w_loss, search_w_top1, search_w_top5 = search_func(search_loader, network, criterion, w_scheduler, w_optimizer, a_optimizer, epoch_str, xargs.print_freq, logger)



[Search the 000-050-th epoch] Time Left: [00:00:00], LR=0.025
|nor_conv_1x1~0|+|skip_connect~0|nor_conv_1x1~1|+|nor_conv_1x1~0|avg_pool_3x3~1|avg_pool_3x3~2|
datasets : ['cifar10-valid', 'cifar10', 'cifar100', 'ImageNet16-120'], extra-info : arch-index=4076
cifar10-valid  FLOP= 19.58 M, Params=0.157 MB, latency=16.88 ms.
cifar10-valid  train : [loss = 0.012, top1 = 99.78%], valid : [loss = 0.675, top1 = 85.43%]
cifar10        FLOP= 19.58 M, Params=0.157 MB, latency=17.22 ms.
cifar10        train : [loss = 0.024, top1 = 99.40%], test  : [loss = 0.430, top1 = 89.96%]
cifar100       FLOP= 19.59 M, Params=0.163 MB, latency=16.03 ms.
cifar100       train : [loss = 0.344, top1 = 90.78%], valid : [loss = 1.491, top1 = 63.68%], test : [loss = 1.479, top1 = 64.04%]
ImageNet16-120 FLOP=  4.90 M, Params=0.164 MB, latency=13.82 ms.
ImageNet16-120 train : [loss = 2.249, top1 = 42.03%], valid : [loss = 2.567, top1 = 36.13%], test : [loss = 2.559, top1 = 36.67%]
tensor([[-6.1687e-04,  5.0854e-04,  7

|nor_conv_1x1~0|+|nor_conv_3x3~0|nor_conv_1x1~1|+|nor_conv_1x1~0|avg_pool_3x3~1|avg_pool_3x3~2|
datasets : ['cifar10-valid', 'cifar10', 'cifar100', 'ImageNet16-120'], extra-info : arch-index=13517
cifar10-valid  FLOP= 54.97 M, Params=0.400 MB, latency=19.34 ms.
cifar10-valid  train : [loss = 0.010, top1 = 99.80%], valid : [loss = 0.553, top1 = 87.88%]
cifar10        FLOP= 54.97 M, Params=0.400 MB, latency=18.75 ms.
cifar10        train : [loss = 0.020, top1 = 99.47%], test  : [loss = 0.356, top1 = 91.80%]
cifar100       FLOP= 54.97 M, Params=0.406 MB, latency=19.93 ms.
cifar100       train : [loss = 0.270, top1 = 92.68%], valid : [loss = 1.304, top1 = 67.78%], test : [loss = 1.306, top1 = 67.70%]
ImageNet16-120 FLOP= 13.75 M, Params=0.407 MB, latency=23.12 ms.
ImageNet16-120 train : [loss = 2.051, top1 = 46.41%], valid : [loss = 2.323, top1 = 40.43%], test : [loss = 2.321, top1 = 39.97%]
tensor([[-0.0006,    -inf,  0.0008, -0.0017,  0.0002],
        [   -inf,    -inf,    -inf, -0.0005,

|nor_conv_1x1~0|+|nor_conv_3x3~0|nor_conv_1x1~1|+|nor_conv_1x1~0|avg_pool_3x3~1|avg_pool_3x3~2|
datasets : ['cifar10-valid', 'cifar10', 'cifar100', 'ImageNet16-120'], extra-info : arch-index=13517
cifar10-valid  FLOP= 54.97 M, Params=0.400 MB, latency=19.34 ms.
cifar10-valid  train : [loss = 0.010, top1 = 99.80%], valid : [loss = 0.553, top1 = 87.88%]
cifar10        FLOP= 54.97 M, Params=0.400 MB, latency=18.75 ms.
cifar10        train : [loss = 0.020, top1 = 99.47%], test  : [loss = 0.356, top1 = 91.80%]
cifar100       FLOP= 54.97 M, Params=0.406 MB, latency=19.93 ms.
cifar100       train : [loss = 0.270, top1 = 92.68%], valid : [loss = 1.304, top1 = 67.78%], test : [loss = 1.306, top1 = 67.70%]
ImageNet16-120 FLOP= 13.75 M, Params=0.407 MB, latency=23.12 ms.
ImageNet16-120 train : [loss = 2.051, top1 = 46.41%], valid : [loss = 2.323, top1 = 40.43%], test : [loss = 2.321, top1 = 39.97%]
tensor([[   -inf,    -inf,  0.0008, -0.0017,    -inf],
        [   -inf,    -inf,    -inf, -0.0005,

|nor_conv_1x1~0|+|nor_conv_3x3~0|nor_conv_1x1~1|+|nor_conv_3x3~0|nor_conv_3x3~1|avg_pool_3x3~2|
datasets : ['cifar10-valid', 'cifar10', 'cifar100', 'ImageNet16-120'], extra-info : arch-index=8037
cifar10-valid  FLOP=121.82 M, Params=0.858 MB, latency=21.93 ms.
cifar10-valid  train : [loss = 0.004, top1 = 99.94%], valid : [loss = 0.524, top1 = 89.14%]
cifar10        FLOP=121.82 M, Params=0.858 MB, latency=22.01 ms.
cifar10        train : [loss = 0.007, top1 = 99.87%], test  : [loss = 0.342, top1 = 92.28%]
cifar100       FLOP=121.82 M, Params=0.864 MB, latency=19.83 ms.
cifar100       train : [loss = 0.130, top1 = 97.20%], valid : [loss = 1.334, top1 = 68.80%], test : [loss = 1.311, top1 = 69.36%]
ImageNet16-120 FLOP= 30.46 M, Params=0.866 MB, latency=25.01 ms.
ImageNet16-120 train : [loss = 1.981, top1 = 47.85%], valid : [loss = 2.345, top1 = 40.93%], test : [loss = 2.343, top1 = 41.60%]
tensor([[   -inf,    -inf,  0.0008, -0.0017,    -inf],
        [   -inf,    -inf,    -inf, -0.0005, 